In [4]:
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
import pandas as pd
np.random.seed(1)
#
df1 = pd.read_csv(r"D:\Learn\BJ13_M32x32_T30_In.csv")
df2 = pd.read_csv(r"D:\Learn\BJ14_M32x32_T30_In.csv")
df3 = pd.read_csv(r"D:\Learn\BJ15_M32x32_T30_In.csv")
df4 = pd.read_csv(r"D:\Learn\BJ16_M32x32_T30_In.csv")

array1 = df1.to_numpy()
array2 = df2.to_numpy()
array3 = df3.to_numpy()
array4 = df4.to_numpy()

max_rows = max(array1.shape[0], array2.shape[0], array3.shape[0], array4.shape[0])
max_cols = max(array1.shape[1], array2.shape[1], array3.shape[1], array4.shape[1])

def pad_array(array, shape):
    padded = np.zeros(shape)
    padded[:array.shape[0], :array.shape[1]] = array
    return padded

array1_padded = pad_array(array1, (max_rows, max_cols))
array2_padded = pad_array(array2, (max_rows, max_cols))
array3_padded = pad_array(array3, (max_rows, max_cols))
array4_padded = pad_array(array4, (max_rows, max_cols))

X0 = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X0.shape)
print(type(X0))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [5]:
def dell(array1,missing_rate):
    array2 = array1 * np.round(np.random.rand(array1.shape[0], array1.shape[1]) + 0.5 - missing_rate)
    return array2
array1 = dell(array1,0.5)
array2 = dell(array2,0.5)
array3 = dell(array3,0.5)
array4 = dell(array4,0.5)

In [6]:
array1_dell = pad_array(array1, (max_rows, max_cols))
array2_dell = pad_array(array2, (max_rows, max_cols))
array3_dell = pad_array(array3, (max_rows, max_cols))
array4_dell = pad_array(array4, (max_rows, max_cols))
X = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X.shape)
print(type(X))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [212]:
print(array1.shape)
X=dell(array1,0.5)
print(X.shape)
X0=array1.reshape(4887,32,32)
X=X.reshape(4887,32,32)
X=X[:32,:,:]
print(X.shape)
X0=X0[:32,:,:]

(4887, 1024)
(4887, 1024)
(32, 32, 32)


In [268]:
import tensorly as tl
from tensorly.decomposition import tucker
def Res():
    return 10
def fold(unfolded, shape, mode):
    full_shape = np.array(shape)
    full_shape[mode], full_shape[0] = full_shape[0], full_shape[mode]
    return np.moveaxis(np.reshape(unfolded, full_shape), 0, mode)
def unfold(z,rank):
    return np.reshape(np.moveaxis(z, rank, 0), (z.shape[rank], -1))
def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew
def ell_G(G, S, R, M, X, K,n):
    sum1=np.zeros((32,1024))
    for t in range(X.shape[2]):
        sum1+=K[t,:,:]@S@G@(np.kron(M[:,t].T,R).T)@S.T@K[t,:,:].T@np.kron(M[:,t].T,R)
    return sum1+n*G
def conj_grad_G(G, S, R, M, X, K, n,maxiter = 5):
    rank, dim1 = G.shape
    g=G.reshape(-1, order = 'F')
    B=np.zeros((32,1024))
    for t in range(X.shape[2]):
        B+=K[t,:,:]@X[t,:,:]@S.T@K[t,:,:].T@np.kron(M[:,t].T,R)
    r = np.reshape(B-ell_G(G, S, R, M, X, K,n) ,-1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_G(Q, S, R, M, X, K,n), -1, order = 'F')
        g, r, q, rold = update_cg(g, r, q, Aq, rold)
    return np.reshape(g, (rank, dim1), order = 'F')
def conj_grad_S_R_M(G, S, R, M, X, K,Q1,H,a,b,n ,r1,maxiter = 5):
    def ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        sum1=np.zeros((32,32))
        for t in range(X.shape[2]):
            sum1+=K[t,:,:]@S@G@(np.kron(M[:,t].T,R).T)@K[t,:,:].T@np.kron(M[:,t].T,R)@G.T
            #sum+=K[t,:,:]@S
        return (sum1+r1*S-a*b*H.T*Q1.T+n*S)
    def ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        sum1=np.zeros((1024,1))
        sum1=sum1.reshape(-1,order = 'F')
        sum2=np.zeros((1024,1))
        sum3=np.zeros((1024,1))
        sum4=np.zeros((1024,1))
        for t in range(X.shape[2]):
            #print(G.shape)
            #print((np.kron(M[:,t].T,np.eye(R.shape[0]))).shape)
            #print((K[t,:,:]@X[t,:,:]-K[t,:,:]@S@G@(np.kron(M[:,t],np.eye(R.shape[0]))).T).shape)
            sum2=np.reshape(K[t,:,:]@X[t,:,:]-K[t,:,:]@S@G@(np.kron(M[:,t],np.eye(R.shape[0]))).T,-1,order = 'F')
            sum3=np.reshape(np.kron(M[:,t],np.eye(R.shape[0]))@G.T@S.T@K[t,:,:].T,-1,order = 'F')
            #print("sum2 shape:", sum2.shape)
            #print("reshaped R.T shape:", np.reshape(R.T, -1, order='F').shape)
            #print("sum3 shape:", sum3.shape)
            #print("term shape:", (sum2 * np.reshape(R.T, -1, order='F') * sum3).shape)
            #print("reshaped n * R shape:", np.reshape(n * R, -1, order='F').shape)
            sum4=sum2*np.reshape(R.T,-1, order = 'F')*sum3+np.reshape(n*R,-1,order = 'F')
            #print("sum4 shape:", sum4.shape)
            #print("sum1 shape:", sum1.shape)
            sum1+=sum4
            #print("sum1 shape:", sum1.shape)
        return sum1


    def con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        rank, dim1 = S.shape
        s=S.reshape(-1, order = 'F')
        B=np.zeros((32,32))
        for t in range(X.shape[2]):
            B+=K[t,:,:]@X[t,:,:]@K[t,:,:].T@np.kron(M[:,t].T,R)@G.T
        r=np.reshape(B-ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, Q, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            s, r, q, rold = update_cg(s, r, q, Aq, rold)        
        return np.reshape(s, (rank, dim1), order = 'F')
    
    def con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        rank, dim1 = R.shape
        rr=S.reshape(-1, order = 'F')
        B=np.zeros((32,32))
        B+=r1*(np.eye(R.shape[0])-a*b*H.T@Q1.T)
        B=B.reshape(-1, order = 'F')
        r=np.reshape(B-ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_R(G, S,Q, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            rr, r, q, rold = update_cg(rr, r, q, Aq, rold)        
        return np.reshape(rr, (rank, dim1), order = 'F')
    
    def ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        sum1=np.zeros((M.shape[0],M.shape[1]))
        sum2=np.zeros((M.shape[0]*M.shape[1],1))
        sum3=np.zeros((M.shape[0]*M.shape[1],1))
        sum4=np.zeros((M.shape[0],M.shape[1]))
        I=np.reshape(np.ones((M.shape[0], 1)),-1,order = 'F')
        for t in range(X.shape[2]):
            #print(G.shape)
            #print((np.kron(M[:,t].T,np.eye(R.shape[0]))).shape)
            #print((K[t,:,:]@X[t,:,:]-K[t,:,:]@S@G@(np.kron(M[:,t],np.eye(R.shape[0]))).T).shape)
            sum2=K[t,:,:]@X[t,:,:]-K[t,:,:]@S@G@(np.kron(I,R.T)).T
            sum3=np.kron(I,R.T)@G.T@S.T@K[t,:,:].T
            #print("sum2 shape:", sum2.shape)
            #print("reshaped R.T shape:", np.reshape(R.T, -1, order='F').shape)
            #print("sum3 shape:", sum3.shape)
            #print("term shape:", (sum2 * M[:,t] * sum3).shape)
            sum4=sum2*M[:,t]*sum3+n*M
            #print("sum4 shape:", sum4.shape)
            sum1+=sum4
            #print("sum1 shape:", sum1.shape)
        return sum1
    def con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        rank, dim1 = M.shape
        m=M.reshape(-1, order = 'F')
        B=np.zeros((32,32))
        B=np.eye(R.shape[0])-a*b*H.T@Q1.T
        r=np.reshape(B-ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_R(G, S,R, Q, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            m, r, q, rold = update_cg(m, r, q, Aq, rold) 
        return np.reshape(m, (rank, dim1), order = 'F')
    S=con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)
    R=con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)
    M=con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)
    return S,R,M

                                                                                           
                                                                                           
                                                                                           
def conj_grad_Q_H(S,R,M,Q1, H, a, b,maxiter = 5):
    def ell_Q1(Z,Q, H, a, b):       
        return (a*b*Q@H@Z@Z.T@H.T)
    def conj_grad_Q(Q1, H, S,R,M, a, b,maxiter = 5):
        Z=M
        rank, dim1 = Q1.shape
        q1=Q1.reshape(-1, order = 'F')
        r=np.reshape(a*b*Z@Z.T@H.T-ell_Q1(Z,Q1, H, a, b) ,-1, order = 'F')
        
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_Q1(Z,Q, H, a, b), -1, order = 'F')
            q1, r, q, rold = update_cg(q1, r, q, Aq, rold) 
        return np.reshape(q1, (rank, dim1), order = 'F')
    def ell_H(Z,Q1, H, a, b):
        return (a*b*Q1@H@Z@Z.T@Q1.T)
    def conj_grad_H(Q1, H, S,R,M, a, b,maxiter = 5):
        Z=M
        rank, dim1 = H.shape
        h=H.reshape(-1, order = 'F')
        r=np.reshape(a*b*Z@Z.T@Q1.T-ell_H(Z,Q1, H, a, b) ,-1, order = 'F')
        
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_H(Z,Q, H, a, b), -1, order = 'F')
            h, r, q, rold = update_cg(h, r, q, Aq, rold) 
        return np.reshape(h, (rank, dim1), order = 'F')
    Q1=conj_grad_Q(Q1, H, S,R,M, a, b)
    H=conj_grad_H(Q1, H, S,R,M, a, b)
    return Q1,H           
def conj_grad_a_b(Q, H, Z, a, b,maxiter = 5):
    def ell_a(Q, H, Z, a, b):
        return (a*b*b*Q@H)
    def ell_b(Q, H, Z, a, b):
        return (a*a*b*Q@H)
    def conj_grad_a(Q1, H, Z, a, b,maxiter = 5):
        I=np.eye(Q1.shape[0])
        r=np.reshape(b*I-ell_a(Q1,H, Z,a, b) ,-1, order = 'F')        
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = q[0]
            Aq = np.reshape(ell_a(Q1,H, Z,Q, b), -1, order = 'F')
            a, r, q, rold = update_cg(a, r, q, Aq, rold) 
        return a[0]
    def conj_grad_b(Q1, H, Z, a, b,maxiter = 5):
        I=np.eye(Q1.shape[0])
        r=np.reshape(a*I-ell_b(Q1, H,Z, a, b) ,-1, order = 'F')        
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = q[0]
            Aq = np.reshape(ell_b(Q1, H, Z, a, Q), -1, order = 'F')
            b, r, q, rold = update_cg(b, r, q, Aq, rold) 
        return b[0]
    a=conj_grad_a(Q, H, Z, a, b)
    b=conj_grad_b(Q, H, Z, a, b)
    return a,b
def compute_rse(var, var_hat):
    return np.linalg.norm(var - var_hat, 2) / np.linalg.norm(var, 2)
def compute_rse1(dense_tensor, pos_test, model_predictions):
    # 提取实际值和预测值
    actual_values = dense_tensor[pos_test]
    predicted_values = model_predictions[pos_test]
    
    # 计算RSE
    rse = np.linalg.norm(actual_values - predicted_values, 2) / np.linalg.norm(actual_values, 2)
    return rse

def Star(X0,X,rank,n,r,rthers,maxiter):
    #Create K
    K = np.zeros(X.shape) 
    K[(X0 != 0) & (X == 0)] = 1
    pos_test = np.where((X0 != 0) & (X == 0))
    #Initialize G S R M Q H A B randomly
    G = np.random.rand(rank, rank, rank)
    S = np.random.rand(X.shape[2], rank)
    R = np.random.rand(X.shape[1], rank)
    M = np.random.rand(rank,X.shape[0])
    print(G.shape,S.shape,R.shape,M.shape)
    Q = np.random.rand(X.shape[0], X.shape[0])
    H = np.random.rand(X.shape[0], X.shape[0])
    a = np.random.uniform(0, 1)
    b = np.random.uniform(0, 1)
    #iter
    show_iter=10
    for iter in range(maxiter):
        for k in range(1):
            print(iter)
            Gshape=G.shape
            G=unfold(G,k)
            #print(K.shape,X.shape,G.shape)
            G = conj_grad_G(G, S, R, M, X, K,n)
            S, R, M = conj_grad_S_R_M(G, S, R, M, X, K,Q,H,a,b,n ,r)
            
            Q, H = conj_grad_Q_H(S,R,M,Q, H, a, b)
            a, b = conj_grad_a_b(Q, H,R ,a, b)
            G=fold(G,Gshape,k)
            print(a,b)
        if (iter + 1) % show_iter == 0:
            #t1 = np.einsum('ijh, ki -> kjh', G, S)
            #t2 = np.einsum('ijh, kj -> ikh', t1, R)
            #t3 = np.einsum('ijh, kh -> ijk', t2, M)
            #temp_hat = t3[pos_test]
            print('Iter: {}'.format(iter + 1))
            #print(compute_rse1(X,pos_test, t3))   
        if Res()<thers:
            break
            
    return G,S,R,M,Q,H,a,b

In [271]:
rank=32
thers=0.001
maxiter=5
n=0.1
r=0.1
G,S,R,M,Q,H,a,b=Star(X0,X,rank,n,r,thers,maxiter)
print(G.shape,S.shape,R.shape,M.shape)
print(S,R,M)
t1 = np.einsum('ijh, ki -> kjh', G, S)
t2 = np.einsum('ijh, kj -> ikh', t1, R)
t3 = np.einsum('ijh, kh -> ijk', t2, M)

print(t3.shape)

    #rank=[50,50,50]
    #core, factors = tucker(X, rank)
    #G = core
    #S = factors[0]
    #R = factors[1]
    #M = factors[2]

(32, 32, 32) (32, 32) (32, 32) (32, 32)
0
-2.939207852204118e+17 1.3411094797961054
1
-3.280301285552011e+19 -72.80572860245627
2
1.2656035298593265e+21 -3993.203952114284
3
1.4281431267257096e+23 6350.4176293645
4
-1.3551140572822054e+25 nan
(32, 32, 32) (32, 32) (32, 32) (32, 32)
[[-2.58218763e+10  1.16526676e+10  1.57012365e+10 ...  6.44781546e+09
  -1.64569163e+10 -1.02683806e+10]
 [-6.59688673e+10 -8.51762084e+10 -1.13817781e+11 ... -5.94146165e+10
  -6.97641934e+09 -3.19809161e+10]
 [-6.34706751e+10 -9.29913768e+10 -1.36592587e+11 ... -7.11232818e+10
  -5.65651430e+09 -2.44410594e+10]
 ...
 [-2.10863298e+10 -3.45153046e+10 -3.87340229e+10 ... -2.91994644e+10
  -1.06308911e+09  8.60614391e+07]
 [ 6.82671889e+10  1.17439963e+11  1.40773395e+11 ...  7.61651074e+10
  -1.28462714e+10  3.67858842e+10]
 [ 1.56555913e+10  2.23718815e+10  1.16157270e+10 ...  8.92293951e+09
  -8.94555872e+09 -8.34998579e+08]] [[-4.64976429e+10  1.15704808e+10  1.67289305e+10 ...  6.75856731e+09
  -1.024841

E:\Anaconda\envs\BERT3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
E:\Anaconda\envs\BERT3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in add
  app.launch_new_instance()


In [272]:
print(t3)

[[[ 8.87252639e+30  2.67334259e+30  1.38511313e+31 ...  9.02692732e+30
    7.77193755e+30  9.71654966e+30]
  [ 1.09631542e+32  2.17553394e+31  1.66359149e+32 ...  1.06329720e+32
    9.62129247e+31  1.13002514e+32]
  [ 1.09343019e+32  2.16501139e+31  1.65651512e+32 ...  1.06015403e+32
    9.60104332e+31  1.12306956e+32]
  ...
  [ 3.34403948e+31  6.42854185e+30  5.07848217e+31 ...  3.22131261e+31
    2.95743804e+31  3.41525865e+31]
  [-1.51131640e+32 -2.96320899e+31 -2.29268146e+32 ... -1.46204512e+32
   -1.32793221e+32 -1.55794041e+32]
  [-3.25923400e+31 -6.44370818e+30 -4.99144456e+31 ... -3.15411566e+31
   -2.86951669e+31 -3.38504340e+31]]

 [[ 7.29988914e+30 -9.62704610e+30 -1.21813522e+31 ...  1.02373255e+31
   -8.52195373e+30  1.06476884e+31]
  [ 2.59362029e+31 -1.72282962e+32 -2.62569285e+32 ...  6.27265951e+31
   -1.95151796e+32  4.05842323e+31]
  [ 8.39492357e+30 -1.81971317e+32 -2.88970670e+32 ...  4.29538502e+31
   -2.09466094e+32  1.81821130e+31]
  ...
  [-3.97749997e+30 -5.7